# Parser tests

In [1]:
[global]
from dsc.dsc_parser import DSC_Script as ds
from dsc.utils import FormatError

def test_format(obj, *args):
    try:
        return obj(*args)
    except Exception as e:
        assert type(e).__name__ == 'FormatError'
        print(e)
        return None

## Basic syntax

In [2]:
%run
[1]
# F: missing output
text = '''
simulate: sim.R
    x: 1
    y: 2
'''
test_format(ds, text)

Cannot find required section ``DSC``!


In [3]:
%run
[2]
# F: grouped parameters is not allowed
text = '''
simulate: sim.R
    group_1:
        x: 1
        y: 2
DSC:
    run: simulate
'''
test_format(ds, text)

Invalid decoration ``group_1``. Decorations must start with ``@`` symbol.


In [4]:
%run
[3]
# P: exec decoration
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1]), ('y', [2])]

In [5]:
%run
[4]
# F: grouped parameters is not allowed in exec decoration
text = '''
simulate: sim.R
    @simulate:
        group_1:
            x: 1
            y: 2
DSC:
    run: simulate
'''
res = test_format(ds, text)

Invalid decoration ``group_1``. Decorations must start with ``@`` symbol.


In [6]:
%run
[5]
# F: alias is not a list
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @ALIAS:
        x_1: x
DSC:
    run: simulate
'''
res = test_format(ds, text)

Cannot find module ``simulate`` in @ALIAS specification ``['x_1']``.


In [1]:
%run
[6]
# P: alias for specific module, both syntax supported
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @ALIAS: 
        simulate: x_1 = x
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('y', [2]), ('x_1', [1])]

text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS: x_1 = x
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('y', [2]), ('x_1', [1])]

text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
        @ALIAS: 
            *: x_1 = x
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('y', [2]), ('x_1', [1])]

In [8]:
%run
[7]
# F: invalid decoration / module
text = '''
simulate: sim.R
    @simulate:
        x: 1
        y: 2
    @test:
        x: 3
DSC:
    run: simulate
'''
res = test_format(ds, text)

Undefined decoration ``@test``.


In [9]:
%run
[8]
# F: invalid decoration / module
text = '''
simulate: sim.R
    @simulate, test:
        x: 1
        y: 2
DSC:
    run: simulate
'''
res = test_format(ds, text)

Undefined decoration ``@test``.


In [10]:
%run
[9]
# P: use global variable
text = '''
simulate: sim.R
    x: 1
    y: $(x)
DSC:
    run: simulate
    global:
        x: 2
'''
res = test_format(ds, text)
list(res.modules['simulate'].dump()['input'].items()) == [('x', [2]), ('y', [2])]
text = '''
simulate: sim.R
    x: 1
    y: $(x)[3]
DSC:
    run: simulate
    global:
        x: 1,2,3,4
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].dump()['input'].items()) == [('x', [1,2,3,4]), ('y', [3])]

In [11]:
%run
[10]
# P: alias partial list / dict
text = '''
simulate: sim.R
  x: 1
  y: 2
  @ALIAS: args = List(), arg1 = List(xvar = x, yy = y)
DSC:
    run: simulate
'''
res = test_format(ds, text)
assert list(res.modules['simulate'].plugin.dump()['container_variables']) == ['x', 'y', 'xvar = x', 'yy = y']

In [12]:
%run
[11]
# P: filter decorator
text = '''
simulate: sim.R
    x: R(1:5)
    @FILTER:
        *: x < 3
DSC:
    run: simulate
'''
res = test_format(ds, text)

## Bad variable / module names

In [4]:
%run
[12]
# F: various bad names
text = '''
@@simulate: sim.R
    x: 1
'''
res = test_format(ds, text)
text = '''
$simulate: sim.R
    x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    .x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    _x: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x$: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x.y: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x_y: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    x.1: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    1: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    _: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    **: 1
'''
res = test_format(ds, text)
text = '''
simulate: sim.R
    .: 1
'''
res = test_format(ds, text)

Invalid variable name ``@@simulate``
Invalid variable name ``$simulate``
Dot is not allowed for module / variable names, in ``.x``. Note that dotted names is not acceptable to Python and SQL. If this limitation is irrelevant to your problem, and you'd rather not rename variable in your code, then please rename ``.x`` to, eg, ``name`` in DSC and use ``@ALIAS: .x = name``.
Names cannot start with underscore, in ``_x``. Note that such naming convention is not acceptable to R. If this limitation is irrelevant to your problem, and you'd rather not rename variable in your code, then please rename ``_x`` to, eg, ``name`` in DSC and use ``@ALIAS: _x = name``.
Invalid variable name ``x$``
Dot is not allowed for module / variable names, in ``x.y``. Note that dotted names is not acceptable to Python and SQL. If this limitation is irrelevant to your problem, and you'd rather not rename variable in your code, then please rename ``x.y`` to, eg, ``name`` in DSC and use ``@ALIAS: x.y = name``.
Cannot 